# PHYS3116

## Stellar Model

A stellar model is a set of computer programs that calculates the interior structure of a star. It does this by integrating the stellar structure equations to calculate the radius, luminosity, pressure, and temperature across a range of mass points from the centre of a star to its surface. For a star with a given mass and composition there is one unique solution for these equations.

We will be constructing a model for a main-sequence star. To build this stellar model we will need a number of ingredients and subroutines. 

Here we will often be using a mass of $M_{tot} = 1.2 M_\odot$ (i.e. 20% more massive than the Sun), and a composition of:
<br><br>$X =$ mass fraction of hydrogen $= 0.7$,
<br>$Y =$ mass fraction of helium $= 0.2$,
<br>$Z =$ mass fraction of all other elements combined $= 0.1$.

Note that in the second assignment you may be asked to vary these to model stars with different masses and compositions.

Our stellar model will be written in python (using jupyter notebook). The code provided here is deliberately not concise, not optimised, and could be improved in many ways. It is provided as a working version such that if you are new to python you will have a stellar model that I will show you how to use. If you are already comfortable with python, you may use this code or develop your own version that you like better.

The first task is to calculate the pressure, density, opacity, and energy generation rate of our star.

### Pressure and Density

The total pressure $P$ is the sum of the gas pressure and the radiation pressure. It is related to the temperature $T$, density $\rho$, and mean molecular weight $\mu$ by the equation:

$P = \frac{R}{\mu}\rho T + \frac{a}{3} T^4$

Where $\frac{R}{\mu}\rho T$ represents the gas pressure, while $\frac{a}{3} T^4$ is the radiation pressure. The mean molecular weight is calculated as $\mu = \left(2X+\frac{3Y}{4}+\frac{Z}{2}\right)^{-1}$. <br>$R$ is the gas constant $8.314472$ J$\cdot$mol$^{-1}\cdot$K$^{-1}$, and $a$ is the radiation constant $7.565767\times10^{-16}$ J$\cdot$m$^{-3}\cdot$K$^{-4}$.

We can rearrange this equation to give the density $\rho$, as it relates to the temperature, total pressure, and stellar composition:

$\rho = \frac{\mu}{RT}(P-\frac{aT^4}{3})$

The ratio of gas pressure to total pressure $\beta$ is important for determining the temperature gradient in the star. Note that gas pressure is most important in low mass stars, while for more massive stars radiation pressure will start to dominate.

We will write some functions to accept input values for $X$, $Y$, $\log(T)$, and $\log(P)$, and return values for $\rho$ and $\beta$.

In [2]:
# Import the libraries we need.

import numpy as np

In [4]:
# Definte physical constants.

R = 8.314472        # gas constant J mol-1 K-1
a = 7.565767E-16    # radiation constant J mol-3 K-4

In [9]:
def mean_molecular_weight(X,Y):
    """
    Calculates the mean molecular weight.
    
    Parameters
    ----------
    X: float
        Mass fraction of hydrogen
    Y: float
        Mass fraction of helium
        
    Returns
    -------
    mu: float
        Mean molecular weight
    """
    
    Z = (1 - X - Y)
    mu = 1/((2*X)+(0.75*Y)+(0.5*Z))
    
    return mu

def density(P,T,mu):
    """
    Calculates the density.
    
    Parameters
    ----------
    P: float
        Total pressure
    T: float
        Temperature
    mu: float
        Mean molecular weight
        
    Returns
    -------
    rho: float
        Density
    """
    
    rho = (mu/(R*T))*(P-(a*T**4)/3)
    
    return rho

def pressure_to_density_beta(X, Y, logT, logP):
    """
    Accepts values for X, Y, log(T), and log(P), and returns values for rho and beta.
    """
    
    mu = mean_molecular_weight(X,Y)
    rho = density(10**logP, 10**logT, mu)
    gas_pressure = (R/mu)*rho*(10**logT)
    beta = gas_pressure/(10**logP)
    
    return rho, beta

Let's test this code by calculating $\rho$ and $\beta$ for two test cases: 

$X=0$, $Y=0.98$, $\log(T)=7.55$, $\log(P)=16.85$
<br>$X=0.7$, $Y=0.05$, $\log(T)=8.55$, $\log(P)=12.85$

In [10]:
rho1, beta1 = pressure_to_density_beta(X=0., Y=0.98, logT=7.55, logP=16.85)
rho2, beta2 = pressure_to_density_beta(X=0.7, Y=0.05, logT=8.55, logP=12.85)
 
print('Rho1: %s, Beta1: %s' %(rho1, beta1))
print('Rho2: %s, Beta2: %s' %(rho2, beta2))

Rho1: 320294978.01444757, Beta1: 0.994354119162539
Rho2: -867114078.0782328, Beta2: -564587.0837460966


### Opacity

Opacity $\kappa$ is a quantity (here with units cm$^2$ g$^{-1}$) that describes how difficult it is for photons to move through a medium. It is a function of stellar composition among many other variables, and it is caused by photons experiencing bound-free, free-free, and electron scattering interactions with electrons in the medium. For this model we will use Kramer's opacity law as an approximate function to calculate opacity:

$\log(\kappa) = \log(\kappa_{bf}+\kappa_{ff}+\kappa_{es})$

Where:

$\kappa_{bf} = \frac{4.34\times10^{25}}{2.82(\rho(1+X))^{0.2}}(1-X-Y)(1+X)\rho T^{-3.5}$

$\kappa_{ff} = 3.68\times10^{22}(X+Y)(1+X)\rho T^{-3.5}$

$\kappa_{es} = 0.2(1.0+X)$

We will now write a function to accept input values for $\log(T)$, $\log(\rho)$, $X$, and $Y$, that returns a value for $\log(\kappa)$.

In [11]:
def opacity(logT, logrho, X, Y):
    T = 10**logT
    rho = 10**logrho
    
    #Kramer's opacity law: kappa is proportional to rho and T^-3.5
    k_bf = (4.34e25*(1-X-Y)*(1+X)*rho*np.power(T,-3.5))/(2.82*np.power(rho*(1+X),0.2))  #bound-free absorption
    k_ff = 3.68e22*(X+Y)*(1+X)*rho*np.power(T,-3.5)                                     #free-free absorption
    k_es = 0.2*(1+X)                                                                    #electron scattering
    
    return np.log10(k_bf + k_ff + k_es)

Let's test this code by calculating $\log(\kappa)$ for $X=0.7$, $Y=-0.2$, $\log(T)=5$, $\log(\rho)=-4$. 

In [20]:
test_opacity = opacity(logT=5, logrho=-4, X=0.7, Y=0.2)
print('log(kappa): %s' %test_opacity)

log(kappa): 3.673275090407306


Recall that this is currently in CGS units (where $\kappa$ has units of cm$^2$ g$^{-1}$). To convert to SI units we will need to divide $\kappa$ by 10, or rather subtract 1 from $\log(\kappa)$. I will calculate this below but you may wish to incorporate this directly into the opacity function instead.

In [22]:
SI_test_opacity = test_opacity - 1
print('log(kappa) SI units: %s' %SI_test_opacity)

log(kappa) SI units: 2.673275090407306


### Energy Generation

The energy generation rate $\epsilon$ is the sum of the energy generated by the proton-proton chain and the CNO cycle, both of which convert hydrogen into helium. Our model is slightly simplified to only include the most common reaction pathways in the proton-proton chain and the CNO cycle and to ignore some details of the shape of the Gamow peak, so we can write:

$\epsilon_{pp} = 0.241\times\rho X^2T_6^{-2/3}e^{-33.8T_6^{-1/3}}$

and

$\epsilon_{CNO} = 8.67\times10^{20}\times\rho X X_{CNO}T_6^{-2/3}e^{-152.28T_6^{-1/3}}$

Where $T_6$ is the temperature in millions of K (i.e. $T_6 = T/10^6$ K) and $X_{CNO} = 0.7263\times Z$ is the total mass fraction of carbon, nitrogen, and oxygen.

The proton-proton chain equation comes from Equation 10.46 in Carroll & Ostlie, though here we neglect three correction terms that are each approximately unity. The CNO cycle equation comes from Equation 10.58 in Carroll & Ostlie, though here we neglect a higher-order correction term.

Now we can write a function that accepts input values for $\log(\rho)$, $\log(T)$, $X$, and $Y$, that returns a value for the energy generation rate $\epsilon = \epsilon_{pp}+\epsilon_{CNO}$.

In [23]:
def energy_generation(logrho,logT,X,Y):
    t6 = (10**logT)/(1E6)
    rho = 10**logrho
    Xcno = 0.7263*(1-X-Y)
    
    eps_pp = 0.241*rho*np.power(X,2)*np.power(t6,-2/3)*np.exp(-33.8/np.power(t6,1/3))
    eps_CNO = 8.67E20*rho*X*Xcno*np.power(t6,-2/3)*np.exp(-152.28/np.power(t6,1/3))

    eps = eps_pp + eps_CNO
    
    return eps

Let's test this code by calculating $\epsilon$ for $\log(\rho)=5$, $\log(T)=7$, $X=0.8$, $Y=0.15$.

In [24]:
test_energy_gen = energy_generation(logrho=5,logT=7,X=0.8,Y=0.15)
print(test_energy_gen)

0.0005106973617412187


## More to come next week!